# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary classifier.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use a random forest regressor, as well as another classifier of your choice; either logistic regression, SVM, or KNN. 

- **Question**: Why would we want this to be a classification problem?
- **Answer**: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

The URL here has many query parameters

- `q` for the job search
- This is followed by "+20,000" to return results with salaries (or expected salaries >$20,000)
- `l` for a location 
- `start` for what result number to start on

## Import Libraries

In [ ]:
import requests
from bs4 import BeautifulSoup

Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a `nobr` element inside of a `td` element with `class='snip`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element="jobTitle`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 

### Write 4 functions to extract each item: location, company, job, and salary.

example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- **Make sure these functions are robust and can handle cases where the data/field may not be available.**
    - Remember to check if a field is empty or `None` for attempting to call methods on it
    - Remember to use `try/except` if you anticipate errors
- **Test** the functions on the results above and simple examples

Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

#### Complete the following code to collect results from multiple cities and starting points. 
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

## Import Data

In [ ]:
max_results_per_city = 4000 

results = []

for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
    'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
    'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'Washington+City%2C+DC']):
    for start in range(0, max_results_per_city, 10):
        r = requests.get('http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=' + city + '&jt=fulltime&explvl=entry_level&start=' + str(start))
        b = BeautifulSoup(r.text)
        results.append(b.find_all("div", class_ = "result"))
        

In [ ]:
# Data in results is a list of lists. To make processing easier, I flatten to an unnested list.
results = [item for sublist in results for item in sublist]

## Extraction 

In [ ]:
'''Crafting from "results" a dictionary ("extracted") of desired elements (job title, company, salary, location,
and a short job summary).'''

extracted = {}

def extraction(results):
    titles = []
    companies = []
    salaries = []
    locations = []
    summaries = []
    for result in results:
        
        # job title
        titles.append(result.find('a',{'data-tn-element':'jobTitle'}).text)
        
        # company
        try:
            companies.append(result.find("span",{'class':'company'}).get_text(strip=True))
        except AttributeError:
            companies.append(None)
            
        # salary    
        try:
            salaries.append(result.find(name='nobr').text)
        except AttributeError:
            salaries.append(None)
     
        # location
        locations.append(result.find("span", class_ = "location").text)
        
        # summary
        summaries.append(result.find("span", class_ = "summary").get_text(strip=True))
        
    extracted['titles'] = titles
    extracted['companies'] = companies
    extracted['salaries'] = salaries
    extracted['locations'] = locations
    extracted['summaries'] = summaries

In [ ]:
import pandas as pd

In [ ]:
listings = pd.DataFrame.from_dict(extracted)

# I pulled over 90,000 job postings from Indeed!
listings.shape

In [ ]:
'''Since I'm working with a big file that takes > an hour to scrape, best to save the scraped data on my computer 
(to avoid having to rescrape if I have issues with my kernel).'''

listings.to_csv('/users/nick/desktop/indeed_jobs.csv', encoding='utf-8')

In [ ]:
listings = pd.read_csv('/users/nick/desktop/indeed_jobs.csv')
listings.drop('Unnamed: 0', axis=1, inplace=True)

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [ ]:
'''Unfortunately, most of the job postings did not include salary information. Since these rows won't be any good for 
model-building, I'll drop them.'''

print listings[listings.salaries.isnull()].shape
listings.dropna(subset=['salaries'], how='all', inplace = True)

In [ ]:
# Getting rid of non-annual (hourly, weekly, monthly) salaries.
listings = listings[~listings.salaries.str.contains('month') & ~listings.salaries.str.contains('week') & ~listings.salaries.str.contains('hour')]

In [ ]:
# Another issue to be wary of: a single job that was posted multiple times. How many are there? (I want them dropped.)
print listings.duplicated().sum()

# Out of curiosity -- which jobs are reposted?
listings.loc[listings.duplicated(keep=False), :]

In [ ]:
# Dropping duplicates
listings.drop_duplicates(inplace=True)

# Necessary for iterating via 'enumeration' (see below).
listings.reset_index(drop=True, inplace=True)

Lastly, we need to clean up salary data. 

1. Only a small number of the scraped results have salary information - only these will be used for modeling.
1. Some of the salaries are not yearly but hourly or weekly, these will not be useful to us for now
1. Some of the entries may be duplicated
1. The salaries are given as text and usually with ranges.

#### Find the entries with annual salary entries, by filtering the entries without salaries or salaries that are not yearly (filter those that refer to hour or week). Also, remove duplicate entries

In [ ]:
# Getting rid of commas and $s in the salaries column.
listings.salaries = listings.salaries.apply(lambda x: x.replace(',',''))
listings.salaries = listings.salaries.apply(lambda x: x.replace('$',''))

In [ ]:
import re

In [ ]:
''' Creating a list separate from my dataframe only to insert it into my dataframe may be an unnecessary step,
(instead of making a list, why not make a new column -- or better, overwrite the source column?), but it's 
easy enough and I know how to do it. '''
salaries = []
for salary in listings.salaries:
    salaries.append(re.findall(r'\d+', salary))
    
listings['salary'] = salaries

In [ ]:
''' Checking data type of entries in 'salary' column. If I don't include the inside for loop, the type of each 
cell would be 'list.' ''' 
for i, e in enumerate(listings.salary):
    for a, b in enumerate(listings.salary[i]):
        print type(listings.salary[i][a])

In [ ]:
# To calculate median salary, I'll need to convert "str" to "int."
for i,e in enumerate(listings.salary):
    for a,b in enumerate(listings.salary[i]):
        listings.salary[i][a] = int(b)

In [ ]:
'''Every value, whether a pair of values or a single value, is within a list. But by performing an operation on the 
values inside the list a single (non-list) value is returned.'''
for i,e in enumerate(listings.salary):
    if len(e) > 1:
        listings.salary[i] = sum(e)/2.
    else:
        listings.salary[i] = sum(e)

In [ ]:
# 'salaries', which provides the text from which 'salary' was derived, won't be used in modeling and can be dropped.
listings.drop('salaries', axis=1, inplace=True)

In [ ]:
# Since the format is "City, STATE", this returns city. 
listings['city'] = listings.locations.apply(lambda x: x.split(',')[0])

#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

We could also perform Linear Regression (or any regression) to predict the salary value here. Instead, we are going to convert this into a _binary_ classification problem, by predicting two classes, HIGH vs LOW salary.

While performing regression may be better, performing classification may help remove some of the noise of the extreme salaries. We don't have to choice the `median` as the splitting point - we could also split on the 75th percentile or any other reasonable breaking point.

In fact, the ideal scenario may be to predict many levels of salaries, 

In [ ]:
# Set a default value for whether or not a given job's salary is above the median.
listings['above_median'] = 0

for i, e in enumerate(listings.above_median):
    if listings.salary[i] > listings.salary.median():
        listings.above_median[i] = 1
    else:
        listings.above_median[i] = 0

## Predicting salaries using Random Forests + Another Classifier

## Random Forest - Get Dummies

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score, StratifiedKFold, KFold

In [ ]:
X = listings.city
y = listings.above_median

In [ ]:
X = pd.get_dummies(X)
# Since New York has the largest population, I'll drop it -- it acts as 'baseline' for the rest of my dummy population.
X.drop('New York', axis=1, inplace=True)

In [ ]:
cv = KFold(len(y), n_folds=3, shuffle=True, random_state=41)
rfc = RandomForestClassifier()
cross_val_score(rfc, X, y, cv = cv).mean()

In [ ]:
from sklearn.grid_search import GridSearchCV

param_grid = {'n_estimators':[8,10,12,20],
              'criterion': ['gini', 'entropy'],
              'max_features':['sqrt', 'log2', None],
              'min_samples_split':[2,4,6,8],
              'min_samples_leaf': [1,2,3,4]}

gs = GridSearchCV(rfc, param_grid, cv=cv).fit(X, y)

In [ ]:
print gs.best_score_
print gs.best_params_

## Random Forest - LabelEncoder

In [ ]:
''' I'm using labelencoder mainly so that I can plot. (Plotting not viable when city is split over 30+ dummy variables.) 
But it also makes a leaner dataframe, and since I'm using randomforest, it should have no impact on outcome.'''

X = listings.city
y = listings.above_median

from sklearn.preprocessing import LabelEncoder
X = pd.DataFrame(LabelEncoder().fit_transform(X))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.scatter(X, listings.salary, c=y)

In [ ]:
cv = KFold(len(y), n_folds=3, shuffle=True, random_state=41)
rfc = RandomForestClassifier()
cross_val_score(rfc, X, y, cv = cv).mean()

In [ ]:
param_grid = {'n_estimators':[8,10,12,20],
              'criterion': ['gini', 'entropy'],
              'max_features':['sqrt', 'log2', None],
              'min_samples_split':[2,4,6,8],
              'min_samples_leaf': [1,2,3,4]}

gs = GridSearchCV(rfc, param_grid, cv=cv).fit(X, y)

In [ ]:
print gs.best_score_
print gs.best_params_

## Logistic Regression

I have too many classes, with too few observations, to run a logistic regression. That's okay. It would've 
been a pretty hokey solution anyway.

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title 
- or whether 'Manager' is in the title. 
- Then build a new Random Forest with these features. Do they add any value? 


In [ ]:
listings.titles

In [ ]:
# Create two columns. One for jobs with senior keywords, the other for jobs with junior keywords. Set default to 0.
listings['senior'] = 0
listings['entry'] = 0

In [ ]:
for i, e in enumerate(listings.senior):
    if 'Lead' in listings.titles[i]:
        listings.senior[i] = 1
    elif 'Senior' in listings.titles[i]:
        listings.senior[i] = 1
    elif 'Manager' in listings.titles[i]:
        listings.senior[i] = 1
    elif 'Experienced' in listings.titles[i]:
        listings.senior[i] = 1
    else:
        listings.senior[i] = 0

In [ ]:
for i, e in enumerate(listings.entry):
    if 'Junior' in listings.titles[i]:
        listings.entry[i] = 1
    elif 'Entry' in listings.titles[i]:
        listings.entry[i] = 1
    else:
        listings.entry[i] = 0

In [ ]:
# There weren't many jobs that could easily be identified as entry-level or senior-level.
print listings.senior.sum()
print listings.entry.sum()

In [ ]:
plt.scatter(listings.senior, listings.salary, c=y)

In [ ]:
plt.scatter(listings.entry, listings.salary, c=y)

In [ ]:
X = listings.city
X = pd.get_dummies(X)
X.drop('New York', axis=1, inplace=True)
X['senior'] = listings.senior
X['entry'] = listings.entry

In [ ]:
cv = StratifiedKFold(y, n_folds=5, shuffle=True, random_state=41)
rfc = RandomForestClassifier(n_estimators = 20)
cross_val_score(rfc, X, y, cv = cv).mean()

In [ ]:
gs = GridSearchCV(rfc, param_grid, cv=cv).fit(X, y)
print gs.best_score_
print gs.best_params_

# Support Vector Machines

In [ ]:
from sklearn import svm

# Laughably bad score!
svc = svm.SVC(kernel = 'linear')
cross_val_score(svc, X, y, scoring='accuracy').mean()

In [ ]:
# Even worse!! While gridsearching would make things better, this isn't a route worth pursuing.
# Measuring vectors in an entirely categorical space is misguided.
svc = svm.SVC(kernel = 'rbf')
cross_val_score(svc, X, y, scoring='accuracy').mean()

## TF-IDF w/ Logistic Regression

In [ ]:
X = listings.loc[:, ['titles', 'summaries']]
y = listings.above_median

X['alltext'] = X.titles + X.summaries
X.drop(['titles', 'summaries'], axis=1, inplace=True)

In [ ]:
# I don't want to deal with a list of lists, so I use ravel.
X_train.values.ravel()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer()

from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
tvec.fit(X_train.values.ravel())

X_train = pd.DataFrame(tvec.transform(X_train.values.ravel()).todense(),
                   columns=tvec.get_feature_names())
X_test = pd.DataFrame(tvec.transform(X_test.values.ravel()).todense(),
                   columns=tvec.get_feature_names())

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
lr = LogisticRegression()

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
y_pred = lr.predict(X_test)
metrics.accuracy_score(y_pred, y_test)

## TF-IDF w/ SVM

In [ ]:
X = listings.loc[:, ['titles', 'summaries']]
y = listings.above_median
X['alltext'] = X.titles + X.summaries
X.drop(['titles', 'summaries'], axis=1, inplace=True)
X_train.values.ravel()
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer()
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
tvec.fit(X_train.values.ravel())

In [ ]:
X_train = pd.DataFrame(tvec.transform(X_train.values.ravel()).todense(),
                   columns=tvec.get_feature_names())
X_test = pd.DataFrame(tvec.transform(X_test.values.ravel()).todense(),
                   columns=tvec.get_feature_names())

In [ ]:
import numpy as np
svc = svm.SVC()

gamma_range = 10.**np.arange(-5, 2)
C_range = 10.**np.arange(-2, 3)
kernel_range = ['rbf', 'sigmoid', 'linear', 'poly']
param_grid = dict(gamma=gamma_range, C=C_range, kernel=kernel_range)
gs = GridSearchCV(svc, param_grid, cv=10, scoring='accuracy')
gs.fit(X_train, y_train)

In [ ]:
print gs.best_score_
print gs.best_params_
print gs.best_estimator_

## TF-IDF w/ Random Forest

In [ ]:
X = listings.loc[:, ['titles', 'summaries']]
y = listings.above_median
X['alltext'] = X.titles + X.summaries
X.drop(['titles', 'summaries'], axis=1, inplace=True)
X_train.values.ravel()
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer()
tvec.fit(X.values.ravel())

In [ ]:
X = pd.DataFrame(tvec.transform(X.values.ravel()).todense(),
                   columns=tvec.get_feature_names())

In [ ]:
cv = KFold(len(y), n_folds=3, shuffle=True, random_state=41)
rfc = RandomForestClassifier()
cross_val_score(rfc, X, y, cv = cv).mean()

In [ ]:
param_grid = {'n_estimators':[8,10,12,20],
              'criterion': ['gini', 'entropy'],
              'max_features':['sqrt', 'log2', None],
              'min_samples_split':[2,4,6,8],
              'min_samples_leaf': [1,2,3,4]}

gs = GridSearchCV(rfc, param_grid, cv=cv).fit(X, y)

In [ ]:
print gs.best_score_
print gs.best_params_
print gs.best_estimator_

## Same as above, but with city

In [ ]:
X['city'] = listings.city
X.city = pd.DataFrame(LabelEncoder().fit_transform(X.city))

In [ ]:
gs.fit(X, y)

# Unfortunately, just a little bit worse.
print gs.best_score_
print gs.best_params_
print gs.best_estimator_

## Extra Trees Classifier

In [ ]:
X = listings.loc[:, ['titles', 'summaries']]
y = listings.above_median
X['alltext'] = X.titles + X.summaries
X.drop(['titles', 'summaries'], axis=1, inplace=True)
X_train.values.ravel()
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer()
tvec.fit(X.values.ravel(), y)

In [ ]:
X = pd.DataFrame(tvec.transform(X.values.ravel()).todense(),
                   columns=tvec.get_feature_names())

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

cv = KFold(len(y), n_folds=3, shuffle=True, random_state=41)
etc = ExtraTreesClassifier()
cross_val_score(etc, X, y, cv = cv).mean()

In [ ]:
param_grid = {'n_estimators':[8,10,12,20],
              'criterion': ['gini', 'entropy'],
              'max_features':['sqrt', 'log2', None],
              'min_samples_split':[2,4,6,8],
              'min_samples_leaf': [1,2,3,4]}

gs = GridSearchCV(etc, param_grid, cv=cv)

In [ ]:
gs.fit(X, y)

print gs.best_score_
print gs.best_params_
print gs.best_estimator_

#### Random Forest Regressor

Let's try treating this as a regression problem. 

- Train a random forest regressor on the regression problem and predict your dependent.
- Evaluate the score with a 5-fold cross-validation
- Do a scatter plot of the predicted vs actual scores for each of the 5 folds, do they match?

In [ ]:
X = listings.city
X = pd.get_dummies(X)
X.drop('New York', axis=1, inplace=True)
X['senior'] = listings.senior
X['entry'] = listings.entry

y = listings.salary

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
cross_val_score(rfr, X, y, cv = cv).mean()

In [ ]:
param_grid = {'n_estimators':[8,10,12,20],
              'max_features':['sqrt', 'log2', None],
              'min_samples_split':[2,4,6,8],
              'min_samples_leaf': [1,2,3,4]}

gs = GridSearchCV(rfr, param_grid, cv=cv).fit(X, y)
print gs.best_score_
print gs.best_params_

In [ ]:
plt.scatter(gs.predict(X), y)

## Regression, but with TD-IDF'd X (plus location)

In [ ]:
X = listings.loc[:, ['titles', 'summaries']]
y = listings.above_median
X['alltext'] = X.titles + X.summaries
X.drop(['titles', 'summaries'], axis=1, inplace=True)
X_train.values.ravel()
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer()
tvec.fit(X.values.ravel())

In [ ]:
X = pd.DataFrame(tvec.transform(X.values.ravel()).todense(),
                   columns=tvec.get_feature_names())
X['city'] = listings.city
X.city = pd.DataFrame(LabelEncoder().fit_transform(X.city))

In [ ]:
param_grid = {'n_estimators':[8,10,12,20],
              'max_features':['sqrt', 'log2', None],
              'min_samples_split':[2,4,6,8],
              'min_samples_leaf': [1,2,3,4]}

gs = GridSearchCV(rfr, param_grid, cv=cv).fit(X, listings.salary)
print gs.best_score_
print gs.best_params_

In [ ]:
plt.scatter(gs.predict(X), listings.salary)

In [ ]:
X = listings.loc[:, ['titles', 'summaries']]
y = listings.above_median
X['alltext'] = X.titles + X.summaries
X.drop(['titles', 'summaries'], axis=1, inplace=True)
X_train.values.ravel()
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer()
tvec.fit(X.values.ravel())
X = pd.DataFrame(tvec.transform(X.values.ravel()).todense(),
                   columns=tvec.get_feature_names())